In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Base URL
BASE_URL = "https://books.toscrape.com/catalogue/page-{}.html"
book_data = []

# Helper function to convert star rating class to text
def get_star_rating(tag):
    rating_classes = {
        "One": "One",
        "Two": "Two",
        "Three": "Three",
        "Four": "Four",
        "Five": "Five"
    }
    for cls in tag.get("class", []):
        if cls in rating_classes:
            return rating_classes[cls]
    return "No Rating"

# Loop through all pages
page = 1
while True:
    url = BASE_URL.format(page)
    response = requests.get(url)
    if response.status_code != 200:
        break  # No more pages

    soup = BeautifulSoup(response.text, "html.parser")
    books = soup.select("article.product_pod")

    for book in books:
        title = book.h3.a["title"]
        price = book.select_one("p.price_color").text.strip()
        availability = book.select_one("p.instock.availability").text.strip()
        star_rating = get_star_rating(book.select_one("p.star-rating"))

        book_data.append({
            "Title": title,
            "Price": price,
            "Availability": availability,
            "Star Rating": star_rating
        })

    page += 1

# Create DataFrame and export to CSV
df = pd.DataFrame(book_data)
df.to_csv("books.csv", index=False)
print("Scraping complete. Data saved to books.csv.")

Scraping complete. Data saved to books.csv.
